# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json
# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Augusta,-32.5000,137.7667,29.57,25,70,7.82,AU,1735544123
1,1,Ménaka,15.9182,2.4022,20.01,14,5,5.73,ML,1735544124
2,2,Puerto Ayora,-0.7393,-90.3518,23.64,97,78,1.90,EC,1735544126
3,3,Fort Stockton,30.8940,-102.8793,18.20,22,0,5.66,US,1735544127
4,4,Codrington,-38.2667,141.9667,19.93,66,100,6.84,AU,1735544128


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity", tiles = "EsriNatGeo", color = "City", 
    alpha = 0.5, frame_width = 700, frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,Bredasdorp,-34.5322,20.0403,26.51,43,0,0.85,ZA,1735544154
127,127,Tura,25.5198,90.2201,25.14,31,0,2.52,IN,1735544277
134,134,Hermanus,-34.4187,19.2345,25.45,53,0,2.71,ZA,1735544285
167,167,Tezu,27.9167,96.1667,25.33,30,0,1.26,IN,1735544326
222,222,Howrah,22.5892,88.3103,26.00,57,0,3.09,IN,1735544392
316,316,Brisas de Zicatela,15.8369,-97.0419,23.64,61,0,1.98,MX,1735544507
380,380,Fatehpur,25.9333,80.8000,21.56,40,0,3.61,IN,1735544586
398,398,Kokologo,12.1897,-1.8856,21.52,16,0,4.18,BF,1735544608
399,399,Kokologo,12.1897,-1.8856,21.52,16,0,4.18,BF,1735544608
460,460,Acapulco de Juárez,16.8634,-99.8901,22.90,83,0,1.03,MX,1735544684


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,Bredasdorp,ZA,-34.5322,20.0403,43,
127,Tura,IN,25.5198,90.2201,31,
134,Hermanus,ZA,-34.4187,19.2345,53,
167,Tezu,IN,27.9167,96.1667,30,
222,Howrah,IN,22.5892,88.3103,57,
316,Brisas de Zicatela,MX,15.8369,-97.0419,61,
380,Fatehpur,IN,25.9333,80.8000,40,
398,Kokologo,BF,12.1897,-1.8856,16,
399,Kokologo,BF,12.1897,-1.8856,16,
460,Acapulco de Juárez,MX,16.8634,-99.8901,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000 # Directions are for 10,000 meters
params = {
    "categories":"accommodation.hotel", # Found on geoapify.com itself
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Bredasdorp - nearest hotel: Victoria Hotel
Tura - nearest hotel: Hotel Polo Orchid
Hermanus - nearest hotel: Aloe guest house
Tezu - nearest hotel: No hotel found
Howrah - nearest hotel: Sun India Guest House
Brisas de Zicatela - nearest hotel: Casa de Olas
Fatehpur - nearest hotel: No hotel found
Kokologo - nearest hotel: No hotel found
Kokologo - nearest hotel: No hotel found
Acapulco de Juárez - nearest hotel: Hotel del Valle
Freetown - nearest hotel: Formal Leona hotel
Boulsa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
25,Bredasdorp,ZA,-34.5322,20.0403,43,Victoria Hotel
127,Tura,IN,25.5198,90.2201,31,Hotel Polo Orchid
134,Hermanus,ZA,-34.4187,19.2345,53,Aloe guest house
167,Tezu,IN,27.9167,96.1667,30,No hotel found
222,Howrah,IN,22.5892,88.3103,57,Sun India Guest House
316,Brisas de Zicatela,MX,15.8369,-97.0419,61,Casa de Olas
380,Fatehpur,IN,25.9333,80.8000,40,No hotel found
398,Kokologo,BF,12.1897,-1.8856,16,No hotel found
399,Kokologo,BF,12.1897,-1.8856,16,No hotel found
460,Acapulco de Juárez,MX,16.8634,-99.8901,83,Hotel del Valle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity", color = "City", alpha = 0.5, 
    tiles = "OSM", frame_width = 700, frame_height = 500, hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)